<a href="https://colab.research.google.com/github/Kush-Singh-26/Learning-Pytorch/blob/main/MNIST_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [ ]:
train_dataset = torchvision.datasets.MNIST(
    root='./data', train=True, transform=transform, download=True
)

test_dataset = torchvision.datasets.MNIST(
    root='./data', train=False, transform=transform, download=True
)


In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)

        self.fc1 = nn.Linear(in_features=16 * 4 * 4,out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=84)
        self.fc3 = nn.Linear(in_features=84, out_features=10)

    def forward(self, input):
        c1 = F.relu(self.conv1(input))
        s2 = F.max_pool2d(c1, kernel_size=2, stride=(2,2))
        c3 = F.relu(self.conv2(s2))
        s4 = F.max_pool2d(c3, kernel_size=2, stride=2)
        s4 = torch.flatten(s4, 1)
        f5 = F.relu(self.fc1(s4))
        f6 = F.relu(self.fc2(f5))

        output = self.fc3(f6)

        return output

In [ ]:
criterion = nn.CrossEntropyLoss()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)

print(model)

optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 20

CNN(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [ ]:
for epoch in range(num_epochs):
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [1/20], Loss: 0.0830
Epoch [2/20], Loss: 0.0047
Epoch [3/20], Loss: 0.0095
Epoch [4/20], Loss: 0.0217
Epoch [5/20], Loss: 0.0113
Epoch [6/20], Loss: 0.0016
Epoch [7/20], Loss: 0.0001
Epoch [8/20], Loss: 0.1295
Epoch [9/20], Loss: 0.0002
Epoch [10/20], Loss: 0.0373
Epoch [11/20], Loss: 0.0004
Epoch [12/20], Loss: 0.0016
Epoch [13/20], Loss: 0.0010
Epoch [14/20], Loss: 0.0002
Epoch [15/20], Loss: 0.0006
Epoch [16/20], Loss: 0.0009
Epoch [17/20], Loss: 0.0014
Epoch [18/20], Loss: 0.0092
Epoch [19/20], Loss: 0.0000
Epoch [20/20], Loss: 0.0004


In [ ]:
correct = 0
total = 0

In [ ]:
with torch.no_grad(): # disable gradient computation
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)

        # Get predicted class (highest probability)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total:.2f}%')

Accuracy: 98.92%


In [20]:
torch.save(model.state_dict(), "mnist_cnn.pth")